In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
import ast
from util import check_user_exists, lookup_product_name
import model_functions as mf

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate, Dot, BatchNormalization, Dropout
from tensorflow.keras.models import Model

2024-12-01 23:41:51.762691: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-01 23:41:51.766052: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-01 23:41:51.774985: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733096511.790048   15419 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733096511.794326   15419 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-01 23:41:51.811521: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
#Importing datasets
users_final = pd.read_csv('https://raw.githubusercontent.com/ardahk/amex/refs/heads/main/final/users_final_data.csv')
products_final= pd.read_csv('https://raw.githubusercontent.com/ardahk/amex/refs/heads/main/final/products_final_data.csv')
original_products = pd.read_csv('https://raw.githubusercontent.com/ardahk/amex/refs/heads/main/data/products.csv')

### Model 1

Dense128 for user & product tower

In [ ]:
user_input = Input((16,), name='user_input')
item_input = Input((30,), name='item_input')


user_tower = Dense(128, activation='relu')(user_input)
user_tower = BatchNormalization()(user_tower)

item_tower = Dense(128, activation='relu')(item_input)
item_tower = BatchNormalization()(item_tower)

dot_product = Dot(axes=1)([user_tower, item_tower])
model = Model(inputs=[user_input, item_input], outputs=dot_product)
model.compile(optimizer='adam', loss='binary_crossentropy')

#model.summary()

In [ ]:
# Parameters
batch_size = 500
num_epochs = 20

acc, precision = mf.create_labels_and_train(users_final, products_final, model, batch_size, num_epochs)
print(acc)
print(precision)

## Model 2

In [ ]:
user_input = Input((16,), name='user_input')
item_input = Input((30,), name='item_input')

user_tower = Dense(128, activation='relu')(user_input)
user_tower = Dense(64, activation='relu')(user_input)
user_tower = BatchNormalization()(user_tower)

item_tower = Dense(128, activation='relu')(item_input)
item_tower = Dense(64, activation='relu')(item_input)
item_tower = BatchNormalization()(item_tower)

dot_product = Dot(axes=1)([user_tower, item_tower])
model = Model(inputs=[user_input, item_input], outputs=dot_product)
model.compile(optimizer='adam', loss='binary_crossentropy')

#model.summary()

In [ ]:
# Parameters
batch_size = 500
num_epochs = 20

mf.create_labels_and_train(users_final, products_final, model, batch_size, num_epochs, graph=False)

## Model 3
Testing Batch Norm vs Dropout. This model has no Batch Norm. 

In [ ]:
user_input = Input((16,), name='user_input')
item_input = Input((30,), name='item_input')

user_tower = Dense(128, activation='relu')(user_input)
user_tower = Dense(64, activation='relu')(user_input)

item_tower = Dense(128, activation='relu')(item_input)
item_tower = Dense(64, activation='relu')(item_input)

dot_product = Dot(axes=1)([user_tower, item_tower])
model = Model(inputs=[user_input, item_input], outputs=dot_product)
model.compile(optimizer='adam', loss='binary_crossentropy')

# Parameters
batch_size = 500
num_epochs = 20

mf.create_labels_and_train(users_final, products_final, model, batch_size, num_epochs)

#model.summary()

## Model 4
Testing Batch Norm vs Dropout. This model has Dropout Layers.

In [ ]:
user_input = Input((16,), name='user_input')
item_input = Input((30,), name='item_input')

user_tower = Dense(128, activation='relu')(user_input)
user_tower = Dropout(0.3)(user_tower)  # Adding Dropout
user_tower = Dense(64, activation='relu')(user_input)
user_tower = Dropout(0.3)(user_tower)  # Adding Dropout

item_tower = Dense(128, activation='relu')(item_input)
item_tower = Dropout(0.3)(item_tower)  # Adding Dropout
item_tower = Dense(64, activation='relu')(item_input)
user_tower = Dropout(0.3)(user_tower)  # Adding Dropout

dot_product = Dot(axes=1)([user_tower, item_tower])
model = Model(inputs=[user_input, item_input], outputs=dot_product)
model.compile(optimizer='adam', loss='binary_crossentropy')

# Parameters
batch_size = 500
num_epochs = 20

mf.create_labels_and_train(users_final, products_final, model, batch_size, num_epochs)

### Model 5

Used to illustrate model architecture in the final presentation

In [ ]:
user_input = Input((16,), name='user_input')
item_input = Input((30,), name='item_input')

user_tower = Dense(128, activation='relu')(user_input)
user_tower = Dense(64, activation='relu')(user_input)
user_tower = BatchNormalization()(user_tower)

item_tower = Dense(128, activation='relu')(item_input)
item_tower = Dense(64, activation='relu')(item_input)
item_tower = BatchNormalization()(item_tower)


dot_product = Dot(axes=1)([user_tower, item_tower])
model = Model(inputs=[user_input, item_input], outputs=dot_product)
model.compile(optimizer='adam', loss='binary_crossentropy')

#model.summary()

# Parameters
batch_size = 500
num_epochs = 20

mf.create_labels_and_train(users_final, products_final, model, batch_size, num_epochs)

In [ ]:
#Trying different numbers of epochs/batch size

#batch_size = [100, 250, 500, 750, 900]
batch_size = 500
epochs = [5,10,15,20,25,30,35]

for num_epochs in epochs:
    mf.create_labels_and_train(users_final, products_final, model, batch_size, num_epochs)

In [ ]:
#Trying different numbers of batch size

batches = [100, 250, 500, 750, 900]
#batch_size = 500
num_epochs = 20

for batch_size in batches:
    mf.create_labels_and_train(users_final, products_final, model, batch_size, num_epochs)

# Final Models

In [18]:
## BASE MODEL
user_input = Input((16,), name='user_input')
item_input = Input((30,), name='item_input')


user_tower = Dense(128, activation='relu')(user_input)
user_tower = Dense(64, activation='relu')(user_tower)
user_tower = BatchNormalization()(user_tower)

item_tower = Dense(128, activation='relu')(item_input)
item_tower = Dense(64, activation='relu')(item_tower)
item_tower = BatchNormalization()(item_tower)

dot_product = Dot(axes=1)([user_tower, item_tower])
model = Model(inputs=[user_input, item_input], outputs=dot_product)
model.compile(optimizer='adam', loss='binary_crossentropy')

# Parameters
batch_size = 500
num_epochs = 20

acc, precision = mf.create_labels_and_train(users_final, products_final, model, batch_size, num_epochs, graph=False)
print(acc)
print(precision)

#model.summary()

/home/codespace/.python/current/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['user_input', 'item_input']. Received: the structure of inputs=('*', '*')
  warnings.warn(


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


/home/codespace/.python/current/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['user_input', 'item_input']. Received: the structure of inputs=('*', '*')
  warnings.warn(


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
0.46
0.4583333333333333


In [16]:
## Modified1 MODEL
user_input = Input((16,), name='user_input')
item_input = Input((30,), name='item_input')


user_tower = Dense(256, activation='relu')(user_input)
user_tower = Dense(128, activation='relu')(user_tower)
user_tower = BatchNormalization()(user_tower)

item_tower = Dense(256, activation='relu')(item_input)
item_tower = Dense(128, activation='relu')(item_tower)
item_tower = BatchNormalization()(item_tower)

dot_product = Dot(axes=1)([user_tower, item_tower])
model = Model(inputs=[user_input, item_input], outputs=dot_product)
model.compile(optimizer='adam', loss='binary_crossentropy')

# Parameters
batch_size = 500
num_epochs = 20

acc, precision = mf.create_labels_and_train(users_final, products_final, model, batch_size, num_epochs, graph=False)
print(acc)
print(precision)

#model.summary()

/home/codespace/.python/current/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['user_input', 'item_input']. Received: the structure of inputs=('*', '*')
  warnings.warn(


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


/home/codespace/.python/current/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['user_input', 'item_input']. Received: the structure of inputs=('*', '*')
  warnings.warn(


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
0.49
0.47368421052631576


In [17]:
## Modified2 MODEL
user_input = Input((16,), name='user_input')
item_input = Input((30,), name='item_input')


user_tower = Dense(256, activation='relu')(user_input)
user_tower = Dense(32, activation='relu')(user_tower)
user_tower = BatchNormalization()(user_tower)

item_tower = Dense(256, activation='relu')(item_input)
item_tower = Dense(32, activation='relu')(item_tower)
item_tower = BatchNormalization()(item_tower)

dot_product = Dot(axes=1)([user_tower, item_tower])
model = Model(inputs=[user_input, item_input], outputs=dot_product)
model.compile(optimizer='adam', loss='binary_crossentropy')

# Parameters
batch_size = 500
num_epochs = 20

acc, precision = mf.create_labels_and_train(users_final, products_final, model, batch_size, num_epochs, graph=False)
print(acc)
print(precision)

#model.summary()

/home/codespace/.python/current/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['user_input', 'item_input']. Received: the structure of inputs=('*', '*')
  warnings.warn(


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


/home/codespace/.python/current/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['user_input', 'item_input']. Received: the structure of inputs=('*', '*')
  warnings.warn(


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
0.5
0.4777777777777778
